## 灰色关联度分析(Grey Relation Analysis，GRA)
- 在一个灰色系统中，我们想要了解其中某个我们所关注的某个项目受其他的因素影响的相对强弱。
- 假设我们知道某一个指标可能是与其他的某几个因素相关的，那么我们想知道这个指标与其他哪个因素相对关系强一些，哪个因素相对关系弱一点。
- 把这些因素排个序，得到一个分析结果，我们就可以知道我们关注的这个指标，与因素中的哪些更相关。
### 操作步骤
#### 1. 确定母序列和子序列
- 母序列：参考序列，每一个因素的序列都可做参考序列
- 子序列：比较序列，也就是需要确立顺序的因素序列，可以理解为除了母序列外的所有序列
#### 2. 归一化(无量纲化)
- 减少数据的绝对数值的差异，将它们统一到近似的范围内，重点关注其变化和趋势。
- 主要方法有：
    - 初值化：就是把这一个序列的数据统一除以最开始的值，由于同一个因素的序列的量级差别不大，所以通过除以初值就能将这些值都整理到1这个量级附近。$$x_i'(k)=\frac{x_i(k)}{x_i(1)}$$
    - 均值化：就是把这个序列的数据除以均值，由于数量级大的序列均值比较大，所以除掉以后就能归一化到1的量级附近。$$x_i'(k)=\frac{x_i(k)}{mean(x_i)}$$
#### 3. 计算灰色关联系数 
$$\zeta _i(k)=\frac{min_imin_k|x_0(k)-x_i(k)|+\rho ·max_imax_k|x_0(k)-x_i(k)|}{|x_0(k)-x_i(k)|+\rho ·max_imax_k|x_0(k)-x_i(k)|},\text{其中}\rho \in [0, 1] $$
- 假设 $\rho=0$，则 $$\zeta _i(k)=\frac{min_imin_k|x_0(k)-x_i(k)|}{|x_0(k)-x_i(k)|}=\frac{Constant}{|x_0(k)-x_i(k)|}$$
    - 可以发现，分子的数值对于所有子序列来说都是一样的。分子的数值实际上就是所有因素的所有维度中，与母序列距离最近的维度上的距离。
    - 实际上，这个系数是与第k个维度上，子序列与母序列的距离(差的绝对值，即l1范数)成反比。即这两个数距离越远，可认为越不相关。
- $\rho ·max_imax_k|x_0(k)-x_i(k)|$这一项对于每个i来说也是一个不变的常数constant，所以可以理解为给上面那个式子的分子分母同时加上某个常数，即 $$\zeta _i(k)=\frac{aConstant+bConstant}{|x_0(k)-x_i(k)|+bConstant}$$
    - 由于分子是距离的全局最小值，这就导致下面的分母必然大于分子，而且，如果分母非常大，曲线距离非常远，那么，$\zeta$接近0； 相反，如果$x_i$和$x_0$在所有维度上的差完全一样，那么分数的值就是1。这样$\zeta$取值范围就是0~1之间，0表示不相关，1表示强关联性。
#### 4. 计算关联系数均值，形成关联序
### 总结
- GRA算法本质上来讲就是提供了一种度量两个向量之间距离的方法，对于有时间性的因子，向量可以看成一条时间曲线，而GRA算法就是度量两条曲线的形态和走势是否相近。
- $\rho$调节不同关联系数之间的差异，换句话说，就是输出的分布，使其可以变得更加稀疏或者紧密。以数学角度要言之，该算法即度量已归一化的子向量与母向量的每一维度的l1-norm距离的倒数之和，并将其映射到0～1区间内，作为子母向量的关联性之度量的一种策略。

In [11]:
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True,precision=3)

In [12]:
data = pd.read_csv('gdp.csv', header=None)
data = data.rename({0:'GDP', 1:'第一产业', 2:'第二产业', 3:'第三产业'},axis=1)
data

,GDP,第一产业,第二产业,第三产业
0,1988,386,839,763
1,2061,408,846,808
2,2335,422,960,953
3,2750,482,1258,1010
4,3356,511,1577,1268
5,3806,561,1893,1352


In [13]:
x_mat = np.array(data)

In [14]:
## 定义函数求解关联度
def gray_analysis(x, rho=0.5):
    # 归一化(均值化)
    x_mean = np.mean(x, axis=0)
    x = x / x_mean
    
    # 提取子序列
    X = x[:, 1:]
    # 提取母序列
    Y = x[:, 0].reshape(X.shape[0], 1)
    
    # 利用公式计算灰色关联度
    abs_x0_xi = np.abs(X - Y)
    a = np.min(abs_x0_xi)
    b = np.max(abs_x0_xi)
    gamma_mat = (a + rho * b) / (abs_x0_xi + rho * b)
    corre_degree = np.mean(gamma_mat, axis=0)
    
    print("子序列中各个指标的灰色关联度分别为：", corre_degree)
    return corre_degree

In [15]:
gray_analysis(x_mat)

子序列中各个指标的灰色关联度分别为： [0.508 0.624 0.757]


array([0.508, 0.624, 0.757])

确定权重

In [16]:
class Topsis:
    def __init__(self, X, **typ):
        # 所有待转换的类型
        x_mat = X.copy()
        ctype = ['cmin', 'cmedian', 'crange']
        if typ:
            # 提取待转换类型及对应的列为一个新字典
            type_dic = dict([(t,typ[t]) for t in ctype if t in typ.keys()])
            position = sum(type_dic.values(),[])

            for col_wait_for_convert in position:
                convert_type = [k for k, v in typ.items() if col_wait_for_convert in v][0]
                current_index = typ[convert_type].index(col_wait_for_convert)
                if convert_type == 'cmedian':
                    x_mat.iloc[:,col_wait_for_convert] = self.positivization(x_mat[col_wait_for_convert], convert_type,typ['best_median'][current_index])
                
                elif convert_type == 'crange':
                    x_mat.iloc[:,col_wait_for_convert] = self.positivization(x_mat[col_wait_for_convert], convert_type,typ['best_range'][current_index])
                
                else:
                    x_mat.iloc[:,col_wait_for_convert] = self.positivization(x_mat[col_wait_for_convert],convert_type)
        else:
            print('无需正向化')
            
        self.x_mat = x_mat
            
    def positivization(self, col ,t, best=None):
        if t == 'cmin':
            posit = col.max() - col
            return posit
        elif t == 'cmedian':
            m = max(abs(col - best))
            posit = 1 - abs(col - best) / m
            return posit
        else:
            posit = col
            t == 'crange'
            a,b = best
            m = max(np.append(a-min(col),max(col)-b))
            x_row = col.shape[0]
            for i in range(x_row):
                if col[i] < a:
                    posit[i] = 1 - (a-col[i]) / m
                elif col[i] > b:
                    posit[i] = 1 - (col[i]-b) / m
                else:
                    posit[i] = 1
            return posit

In [17]:
river_data = pd.read_csv('river.csv',header=None)
river_data.sample(5)

,0,1,2,3
0,4.69,6.59,51.0,11.94
18,3.54,7.27,54.0,3.16
9,6.19,7.27,17.0,7.51
2,9.11,6.31,46.0,8.91
15,7.73,6.14,52.0,15.72


In [18]:
tp = Topsis(X=river_data, cmin=[2], cmedian=[1], best_median=[7], crange=[3], best_range=[[10, 20]])

In [19]:
def score(processed_x, raw_data):
    after_processed = processed_x
    temp_mat = after_processed.copy()
    temp_mat = temp_mat / temp_mat.mean(axis=0)
    main_col = temp_mat.max(axis=1)
    temp_mat.insert(0, column='母序列', value=main_col)
    corr = gray_analysis(np.array(temp_mat))
    col_weight = corr / corr.sum()
    object_score = (raw_data * col_weight).sum(axis=1)
    standar_score = object_score / sum(object_score)
    return standar_score

In [20]:
score(tp.x_mat, river_data)

子序列中各个指标的灰色关联度分别为： [0.646 0.607 0.525 0.647]


0     0.057808
1     0.028172
2     0.055168
3     0.071119
4     0.069670
5     0.057809
6     0.045839
7     0.062283
8     0.031623
9     0.030902
10    0.025951
11    0.045317
12    0.043471
13    0.064533
14    0.040179
15    0.064298
16    0.056917
17    0.053891
18    0.051634
19    0.043416
dtype: float64